In [1]:
pathtorepo = "C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\"

using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include("C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\system.jl")
include("C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\visual_regimes.jl");

  Activating project at `C:\Users\Alex\Desktop\repo\2612\0903\dynamical-systems\env\integrate`


In [2]:
using StaticArrays, DifferentialEquations, DynamicalSystems
E, x, y  = 0..40, 0..1,  0..1
box = E × x × y
using CairoMakie, GLMakie;

In [243]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58; const ΔU0 = 0.305; 

In [244]:
time = 1000; tt = 0; tstep = 0.001; times = [time, tt]
integ_set = (alg = Vern9(), adaptive = false, dt = tstep)

time_int_3d = [100000, 200000]
eps_vec = [1e-10, 0.0, 0.0];

In [245]:
using LinearAlgebra

jac = jacob_TM_(fp[1], p, 0)
eivec = eigvecs(jac)

shift = fp[1] + real(eivec[:, 1])*1e-4
shift

3-element SVector{3, Float64} with indices SOneTo(3):
 8.344928548697018
 0.7384915050866405
 0.4383082474463694

In [250]:
U0 = 0.265;
I0 = -1.706

p = [α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0_1 = shift
u0_2 = output[4][end]

println("last point  pink trajectory:$u0_1")
println("black trajectory:$u0_2")

last point  pink trajectory:[8.344928548697018, 0.7384915050866405, 0.4383082474463694]
black trajectory:[13.617786769845223, 0.6498710704050087, 0.45088709028343976]


In [251]:
output = research_regime_2_traj(TM, u0_1, u0_2, p, times, integ_set,
        t_int_3d = time_int_3d,
        plot = false, onlylastpoint = false, dis_hc = true)

9-element Vector{Any}:
  3-dimensional CoupledODEs
  3-dimensional CoupledODEs
  3-dimensional StateSpaceSet{Float64} with 1000001 points
  3-dimensional StateSpaceSet{Float64} with 1000001 points
 2.014298827064272e-5
 0.00024017972200196834
  Figure()
  3-dimensional StateSpaceSet{Float64} with 1 points
  Vector{ComplexF64}[[-6.39202144621012 + 0.0im, 2.1633886996418656 - 13.783195438835794im, 2.1633886996418656 + 13.783195438835794im]]

In [237]:
fp, ei, _ = fixedpoints(output[1], box, jacob_TM_)

(3-dimensional StateSpaceSet{Float64} with 1 points, Vector{ComplexF64}[[-6.389480274087585 + 0.0im, 2.165716578793199 - 13.779650444011056im, 2.165716578793199 + 13.779650444011056im]], Bool[0])

In [238]:
prev_1 = u0_1
prev_2 = u0_2
println("I0: $I0")
println("previous last point  pink trajectory:$prev_1")
println("previous black trajectory:$prev_2")

I0: -1.7064
previous last point  pink trajectory:[8.345438255268782, 0.7384885795134165, 0.4382939618008538]
previous black trajectory:[8.316131139486899, 0.736049299565973, 0.4378038618077226]


In [239]:
data_1 = output[3]
data_2 = output[4];

In [241]:
# 3d interactive
tstart, tfinish = 1, 500000

f = Figure(resolution = (700, 700))
ax = LScene(f[1, 1], show_axis = true)


scale!(ax.scene, 50, 50, 1)

lines!(data_2[time_int_3d[1]:time_int_3d[2], 3], data_2[time_int_3d[1]:time_int_3d[2], 2],
        data_2[time_int_3d[1]:time_int_3d[2], 1], linewidth = 1.0, color = :black)
        
lines!(data_1[tstart:tfinish, 3], data_1[tstart:tfinish, 2],
        data_1[tstart:tfinish, 1], linewidth = 1.0, color = :deeppink)
        
scatter!(output[8][1][3], output[8][1][2], output[8][1][1], markersize = 10.0, color = :orange)
scatter!(u0_1[3], u0_1[2], u0_1[1], markersize = 10.0, color = :green)

display(GLMakie.Screen(), f)

GLMakie.Screen(...)